# 用函數取代表格

用簡單的神經網路來取代 V

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
from gridworld import GridWorld

In [3]:
blocks={(1,1), (3,3)}
gw = GridWorld(size=(5,5), start=(0,0), exit=(4,4), blocks=blocks)

In [4]:
from ipywidgets import widgets as W
from IPython.display import display

In [7]:
gw_html = W.HTML(value=gw._repr_html_())
gw.restart()

def gw_move(i):
    def func(b=None):
        gw_html.value = gw.move(i)._repr_html_()
    return func

def gw_restart(b=None):
    gw.restart()
    gw_html.value = gw._repr_html_()
buttons = [] 
for i, bn in enumerate(['arrow-right', 'arrow-up', 
                        'arrow-left', 'arrow-down', 'refresh']):
    b = W.Button(icon='fa-'+bn, layout=W.Layout(width='5em'))
    b.on_click(gw_move(i) if i<4 else gw_restart)
    buttons.append(b)

W.HBox([gw_html, W.VBox(buttons)])


## 使用 Q learning

Q 用簡單的神經網路來定義

In [8]:
Q = Sequential()
Q.add(Dense(128, input_shape=((gw.size[0]+2)*(gw.size[1]+2)+4,), activation="relu" ))  # 輸入是 i, j 座標和 a
Q.add(Dense(1, activation="tanh")) # 因為輸出是 +-1
Q.compile(loss='mse',optimizer='sgd', metrics=['accuracy'])

avectors = [[0]* 4 for i in range(4)]
for i in range(4):
    avectors[i][i]=1
def Qfunc(i,j):
    ij = np.zeros( (gw.size[0]+2,  gw.size[1]+2))
    ij[i, j]=1
    ij = list(ij.ravel())
    return np.array([Q.predict(np.array([ij+avectors[a]]))[0,0] for a in range(4)])


def Qupdate(i, j, a, v):
    ij = np.zeros( (gw.size[0]+2,  gw.size[1]+2))
    ij[i, j]=1
    ij = list(ij.ravel())
    return Q.train_on_batch(np.array([ij+avectors[a]]), np.array([[v]]))

Qfunc(1,3)


array([ 0.04643077,  0.09782885, -0.08176234, -0.11994256], dtype=float32)

In [9]:
from random import randint, random, shuffle, choice
from time import sleep
gw_html = W.HTML()
display(gw_html)

def update_VA(gw, Qfunc):
    if gw.A is None:
        gw.A = np.full( (gw.size[0]+2, gw.size[1]+2), -1)
    if gw.V is None:
        gw.V = np.full( (gw.size[0]+2, gw.size[1]+2), 2.)
    for i in range(gw.size[0]):
        for j in range(gw.size[1]):
            Qij = Qfunc(i,j)
            if Qij.min() == 2:
                gw.A[i,j]=-1
                gw.V[i,j]= 2
            else:
                gw.A[i,j] = np.argmax(np.where( Qij > 1, -2, Qij))
                gw.V[i,j] = np.max(np.where( Qij > 1, -2, Qij))
    gw_html.value = gw._repr_html_()

def Qlearn(g):
    actions = [0,1,2,3]
    while True:
        gw_html.value = g._repr_html_()
        if g.is_end():
            break
        sleep(0.02)
        src_pos = g.pos
        Q_src = Qfunc(*src_pos)
        if random()< ϵ:
            a = choice(actions)
        else:
            a = np.argmax(Q_src)            
        g.move(a)    
        dst_pos = g.pos
        r = g.score
        if r:
            v=r
        else:
            v=r+γ*Qfunc(*dst_pos).max()
        Qupdate(*src_pos, a, v)        

α = 0.1
γ = 0.95
ϵ = 5.
for i in range(500):
    gw.restart()
    gw_html.value = gw._repr_html_()
    Qlearn(gw)         
    update_VA(gw, Qfunc)
    ϵ *= 0.99

HTML(value='')

KeyboardInterrupt: 

In [ ]:
## 另外一種網路

Q = Sequential()
Q.add(Dense(128, input_shape=((gw.size[0]+2)*(gw.size[1]+2),), activation="relu" ))  # 輸入是 i, j 
Q.add(Dense(4, activation="tanh")) # 因為輸出是 +-1
Q.compile(loss='mse',optimizer='sgd', metrics=['accuracy']) # 輸出 a

def Qfunc(i,j):
    ij = np.zeros( (gw.size[0]+2,  gw.size[1]+2))
    ij[i, j]=1
    ij = list(ij.ravel())
    return Q.predict(np.array([ij]))[0]


def Qupdate(i, j, a, v):
    ij = np.zeros( (gw.size[0]+2,  gw.size[1]+2))
    ij[i, j]=1
    ij = list(ij.ravel())
    Y = Q.predict(np.array([ij]))
    Y[0][a] = v
    return Q.train_on_batch(np.array([ij]), Y)

Qfunc(1,3)
